# Stage 1: Data Cleaning and Preparation

**Goal:** Clean and structure the dataset from the World Bank Group's Climate Change Data in order to build a predictive ML model for CO2 emissions later.

**Dataset Link:** https://datacatalog.worldbank.org/dataset/climate-change-data

The dataset includes global data from 1990 to 2011 with features like:
- CO2 emissions
- Population data
- GDP, GNI, FDI
- Land usage
- Climate & disasters


In [18]:
import pandas as pd
import numpy as np
!pip install xlrd

# Set display options for better visibility
pd.set_option('display.max_columns', None)

# Load the dataset
#df = pd.read_csv('climate_change_download_0.csv')
df = pd.read_excel('climate_change_download_0.xls')

# View first few rows
df.head()


,Country code,Country name,Series code,Series name,SCALE,Decimals,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
0,ABW,Aruba,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,29.57481,..,..,..,..,..,..,..,..,..,29.57481,..,..,..,..,..,..,..,..,..,..,..
1,ADO,Andorra,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,..,..,..,..,..,..,0,..,..,..,..,..,..,..,..,..,..,..
2,AFG,Afghanistan,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,..,..,..,..,..,..,0,..,..,..,..,..,..,..,..,..,..,..
3,AGO,Angola,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0.208235,..,..,..,..,..,..,..,..,..,0.208235,..,..,..,..,..,..,..,..,..,..,..
4,ALB,Albania,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,4.967875,..,..,..,..,..,..,..,..,..,4.967875,..,..,..,..,..,..,..,..,..,..,..


In [19]:
# Global Data Overview 

# Check basic shape and types
print("Shape:", df.shape)
df.info()

# Check for unique values in key columns
print("Unique Series:", df['Series name'].nunique())
print("Unique Countries:", df['Country name'].nunique())
print("Year Columns:", df.columns[5:])  # Usually year columns start after index 5


Shape: (13512, 28)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13512 entries, 0 to 13511
Data columns (total 28 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Country code  13512 non-null  object
 1   Country name  13512 non-null  object
 2   Series code   13512 non-null  object
 3   Series name   13512 non-null  object
 4   SCALE         13512 non-null  object
 5   Decimals      13512 non-null  object
 6   1990          10017 non-null  object
 7   1991          10017 non-null  object
 8   1992          10017 non-null  object
 9   1993          10017 non-null  object
 10  1994          10017 non-null  object
 11  1995          10017 non-null  object
 12  1996          10017 non-null  object
 13  1997          10017 non-null  object
 14  1998          10017 non-null  object
 15  1999          10017 non-null  object
 16  2000          10017 non-null  object
 17  2001          10017 non-null  object
 18  2002          10017 non-nul

# Initial Project Goals 

**Initial Goals:**
- Remove unnecessary metadata
- Convert series into column-based structure
- Prepare country-year-feature format
- Handle missing/null values
- Prepare a machine learning-ready flat dataset


## Data Cleaning

In [20]:
# Remove rows marked as "Text" in "SCALE" and "Decimals"


df = df[(df['SCALE'] != 'Text') & (df['Decimals'] != 'Text')]

In [21]:
# Drop unnecessary columns


df.drop(columns=['Country name', 'Series code', 'SCALE', 'Decimals'], inplace=True)

In [22]:
# Replace ".." and "" with np.nan
#df.replace({'..': np.nan, '': np.nan}, inplace=True)

pd.set_option('future.no_silent_downcasting', True)
df.replace({'..': np.nan, '': np.nan}, inplace=True)


In [23]:
# Convert year columns to numeric


# Convert all year columns to float
year_columns = df.columns[2:]  # Assuming after "Series name" come the years
df[year_columns] = df[year_columns].apply(pd.to_numeric, errors='coerce')

In [24]:
# Rename features (Series name column)

df['Series name'] = df['Series name'].str.strip().str.replace(' ', '_').str.lower()

# DataFrame Transformation

In [25]:
# Melt the DataFrame


# Convert wide format (years as columns) to long format
df_melted = df.melt(id_vars=['Country code', 'Series name'],
                    var_name='Year',
                    value_name='Value')

# Drop rows with NaN values for now
df_melted.dropna(subset=['Value'], inplace=True)

In [26]:
# Pivot to make each series a column


df_pivot = df_melted.pivot_table(index=['Country code', 'Year'],
                                 columns='Series name',
                                 values='Value').reset_index()

In [27]:
# Fix column names


df_pivot.columns.name = None  # remove the pivot name
df_pivot.head()

,Country code,Year,access_to_electricity_(%_of_total_population),access_to_improved_sanitation_(%_of_total_pop.),access_to_improved_water_source_(%_of_total_pop.),agricultural_land_under_irrigation_(%_of_total_ag._land),annual_freshwater_withdrawals_(%_of_internal_resources),cereal_yield_(kg_per_hectare),"child_malnutrition,_underweight_(%_of_under_age_5)","co2_emissions,_total_(ktco2)",co2_emissions_per_capita_(metric_tons),"co2_emissions_per_units_of_gdp_(kg/$1,000_of_2005_ppp_$)",disaster_risk_reduction_progress_score_(1-5_scale;_5=best),"droughts,_floods,_extreme_temps_(%_pop._avg._1990-2009)",ease_of_doing_business_(ranking_1-183;_1=best),energy_use_per_capita_(kilograms_of_oil_equivalent),"energy_use_per_units_of_gdp_(kg_oil_eq./$1,000_of_2005_ppp_$)","foreign_direct_investment,_net_inflows_(%_of_gdp)",gdp_($),ghg_net_emissions/removals_by_lucf_(mtco2e),gni_per_capita_(atlas_$),invest._in_energy_w/_private_participation_($),invest._in_telecoms_w/_private_participation_($),invest._in_transport_w/_private_participation_($),invest._in_water/sanit._w/_private_participation_($),land_area_below_5m_(%_of_land_area),"malaria_incidence_rate_(per_100,000_people)","methane_(ch4)_emissions,_total_(ktco2e)",nationally_terrestrial_protected_areas_(%_of_total_land_area),"nitrous_oxide_(n2o)_emissions,_total_(ktco2e)","nurses_and_midwives_(per_1,000_people)","other_ghg_emissions,_total_(ktco2e)",paved_roads_(%_of_total_roads),"physicians_(per_1,000_people)",population,population_below_5m_(%_of_total),population_growth_(annual_%),population_in_urban_agglomerations_>1million_(%),population_living_below_$1.25_a_day_(%_of_total),"primary_completion_rate,_total_(%_of_relevant_age_group)",public_sector_mgmt_&_institutions_avg._(1-6_scale;_6=best),ratio_of_girls_to_boys_in_primary_&_secondary_school_(%),"under-five_mortality_rate_(per_1,000)",urban_population,urban_population_growth_(annual_%)
0,ABW,1990,NaN,NaN,100.0,NaN,NaN,NaN,NaN,1840.834,29.620641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.57481,NaN,NaN,0.105547,NaN,NaN,NaN,NaN,NaN,62147.0,29.57482,1.820254,NaN,NaN,NaN,NaN,NaN,NaN,31259.941,1.780501
1,ABW,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1928.842,29.838835,NaN,NaN,NaN,NaN,NaN,NaN,21.185138,8.720670e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.104930,NaN,NaN,NaN,NaN,NaN,64642.0,NaN,3.936181,NaN,NaN,NaN,NaN,NaN,NaN,32321.000,3.337974
2,ABW,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1723.490,25.239288,NaN,NaN,NaN,NaN,NaN,NaN,-3.857809,9.586592e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.104930,NaN,NaN,NaN,NaN,NaN,68286.0,NaN,5.484041,NaN,NaN,NaN,NaN,NaN,NaN,33938.142,4.882234
3,ABW,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1771.161,24.405233,NaN,NaN,NaN,NaN,NaN,NaN,-1.655492,1.083240e+09,NaN,14940.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.104930,NaN,NaN,NaN,NaN,NaN,72573.0,NaN,6.088818,NaN,NaN,NaN,NaN,NaN,NaN,35851.062,5.483368
4,ABW,1994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1763.827,22.977867,NaN,NaN,NaN,NaN,NaN,NaN,-5.874439,1.245810e+09,NaN,15320.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.104930,NaN,NaN,NaN,NaN,NaN,76762.0,NaN,5.611677,NaN,NaN,NaN,NaN,NaN,NaN,37690.142,5.002538


# Handle Missing Values

In [28]:
# Detect % of missing values


missing_percent = df_pivot.isnull().mean().sort_values(ascending=False) * 100
missing_percent[missing_percent > 0]  # View only columns with missing values

disaster_risk_reduction_progress_score_(1-5_scale;_5=best)       98.341232
access_to_electricity_(%_of_total_population)                    98.045024
malaria_incidence_rate_(per_100,000_people)                      97.827804
droughts,_floods,_extreme_temps_(%_pop._avg._1990-2009)          96.682464
annual_freshwater_withdrawals_(%_of_internal_resources)          94.451027
invest._in_water/sanit._w/_private_participation_($)             93.601896
nurses_and_midwives_(per_1,000_people)                           92.575039
agricultural_land_under_irrigation_(%_of_total_ag._land)         91.271722
population_below_5m_(%_of_total)                                 91.034755
land_area_below_5m_(%_of_land_area)                              91.034755
public_sector_mgmt_&_institutions_avg._(1-6_scale;_6=best)       90.955766
child_malnutrition,_underweight_(%_of_under_age_5)               89.652449
invest._in_transport_w/_private_participation_($)                89.218009
ease_of_doing_business_(r

In [29]:
# Drop columns with >50% missing values (adjust as needed)


df_pivot = df_pivot.loc[:, df_pivot.isnull().mean() < 0.5]

In [30]:
# Drop rows with any missing values


df_clean = df_pivot.dropna()

# Export Clean Dataset

In [31]:
df_clean.to_csv("cleaned_climate_data.csv", index=False)
print("Exported cleaned data to climate_clean_data.csv ✅")


Exported cleaned data to climate_clean_data.csv ✅


# Summary
- Cleaned dataset has shape: `(rows, columns)`
- Removed text rows, unneeded columns
- Converted ".." and empty cells to NaN
- Converted wide to long format and pivoted into machine learning-friendly format
- Dropped excessive missing data
- Exported cleaned CSV
